# Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px

# 1. EDA

cargamos el dataset

In [ ]:
df = pd.read_csv('../data/diabetic_data.csv')
df.head()

estudio inicial de los datos

In [ ]:
df.shape

In [ ]:
# los tipos de dato de cada columna
df.dtypes

analisis columna weight

In [ ]:
import pandas as pd

def analisis_valores(columna):

    # Ignorar valores faltantes de la columna original para el conteo
    mask_no_na = ~columna.isna()
    
    # Convertir a numérico, forzando errores a NaN
    numeros = pd.to_numeric(columna, errors='coerce')
    
    # Conteo de valores numéricos y no numéricos (excluyendo nulos originales)
    count_numericos = numeros[mask_no_na].notna().sum()
    count_no_numericos = mask_no_na.sum() - count_numericos
    
    # Obtener los valores no numéricos:
    no_numericos_series = columna[mask_no_na][numeros[mask_no_na].isna()]
    valores_no_numericos = no_numericos_series.unique().tolist()
    
    # Calcular porcentaje de cada valor único no numérico
    conteo_unicos = no_numericos_series.value_counts()
    porcentaje_no_numericos = (conteo_unicos / conteo_unicos.sum() * 100).to_dict()
    
    # Calcular media y desviación estándar para los valores numéricos
    media_numericos = numeros[mask_no_na].dropna().mean()
    std_numericos = numeros[mask_no_na].dropna().std()
    
    return {
        "numericos": count_numericos,
        "no_numericos": count_no_numericos,
        "porcentaje_no_numericos": porcentaje_no_numericos,
        "media_numericos": media_numericos,
        "std_numericos": std_numericos
    }



In [ ]:
analisis_valores(df['weight'])

cambiar ID a objetos y no numeros

In [ ]:
# change wrong data types
df['admission_type_id'] = df['admission_type_id'].astype(str)
df['discharge_disposition_id'] = df['discharge_disposition_id'].astype(str)
df['admission_source_id'] = df['admission_source_id'].astype(str)
df['encounter_id'] = df['encounter_id'].astype(str)
df['patient_nbr'] = df['patient_nbr'].astype(str)


In [ ]:
df.describe()

In [ ]:
import pandas as pd

def estudio_dataset(df, exclude_columns=None):
    """
    Realiza un estudio de las columnas de un dataset.

    Para columnas numéricas, genera un resumen descriptivo (con información adicional sobre valores faltantes).
    Para columnas categóricas, muestra los valores únicos, el porcentaje de cada valor y la cantidad de faltantes.

    Parámetros:
        df (pd.DataFrame): Dataset a analizar.
        exclude_columns (list, opcional): Lista de nombres de columnas a excluir del análisis.

    Retorna:
        dict: Diccionario donde cada clave es el nombre de la columna (no excluida) y el valor es un diccionario
              con el estudio realizado para esa columna.
    """
    resultados = {}
    exclude_columns = exclude_columns or []

    for col in df.columns:
        if col in exclude_columns:
            continue

        if pd.api.types.is_numeric_dtype(df[col]):
            # Estudio para columnas numéricas
            resumen = df[col].describe()
            missing = df[col].isnull().sum()
            resumen_dict = resumen.to_dict()
            resumen_dict['missing'] = missing
            resultados[col] = {
                'tipo': 'numérica',
                'estudio': resumen_dict
            }
        else:
            # Estudio para columnas categóricas
            missing = df[col].isnull().sum()
            unique_values = df[col].dropna().unique().tolist()
            porcentajes = df[col].value_counts(normalize=True, dropna=True) * 100
            resultados[col] = {
                'tipo': 'categórica',
                'unique_values': unique_values,
                'porcentajes': porcentajes.to_dict(),
                'missing': missing
            }
    return resultados



In [ ]:
estudio_dataset(df, exclude_columns=['encounter_id', 'patient_nbr'])

## UNIVARIATE ANALYSIS

In [ ]:
import math
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots

def univariate_analysis_grid(df):
    """
    Genera un grid de plots univariados para todas las columnas del DataFrame.
    
    Para cada columna:
      - Si es numérica: genera un histograma.
      - Si es categórica: genera un gráfico de barras con el conteo de cada categoría.
    
    Los plots se organizan en un grid de 3 columnas por fila.
    
    Parámetros:
        df (pd.DataFrame): Dataset a analizar.
    """
    num_cols = len(df.columns)
    cols_per_row = 3
    rows = math.ceil(num_cols / cols_per_row)
    
    # Crear el grid de subplots
    fig = make_subplots(rows=rows, cols=cols_per_row, subplot_titles=df.columns)
    
    # Iterar por cada columna del DataFrame
    for i, col in enumerate(df.columns):
        row = i // cols_per_row + 1
        col_idx = i % cols_per_row + 1
        
        # Si la columna es numérica, usamos histograma
        if pd.api.types.is_numeric_dtype(df[col]):
            fig_tmp = px.histogram(df, x=col, title=f'Histograma de {col}')
        else:
            # Para columnas categóricas: contar valores
            count_df = df[col].value_counts().reset_index()
            count_df.columns = [col, 'Frecuencia']
            fig_tmp = px.bar(count_df, x=col, y='Frecuencia', title=f'Conteo de {col}')
        
        # Agregar las trazas del plot temporal al subplot correspondiente
        for trace in fig_tmp.data:
            fig.add_trace(trace, row=row, col=col_idx)
    
    # Ajustar layout del grid
    fig.update_layout(height=rows * 400, width=1200, title_text="Análisis Univariado - Grid de Plots")
    fig.show()


In [ ]:
univariate_analysis_grid(df)

# Bivariate analysis


In [ ]:
import pandas as pd
import plotly.express as px

def analisis_numerico(df):
    """
    Función que recibe un DataFrame de pandas y genera un análisis bivariante
    utilizando una matriz de diagramas de dispersión (pair plot) para las columnas numéricas.
    Se ajustan los ángulos de las etiquetas para que no se solapen.
    """
    # Seleccionar únicamente las columnas numéricas
    columnas_numericas = df.select_dtypes(include=['number']).columns
    if len(columnas_numericas) < 2:
        print("El DataFrame debe tener al menos dos columnas numéricas para realizar un análisis bivariante.")
        return

    # Crear la matriz de diagramas de dispersión
    fig = px.scatter_matrix(
        df,
        dimensions=columnas_numericas,
        title="Análisis Bivariante",
        labels={col: col for col in columnas_numericas}
    )

    # Ajustar el tamaño del gráfico y márgenes
    fig.update_layout(width=1300, height=1300, margin=dict(l=100, r=100, t=100, b=100))
    
    # Ajustar los ángulos de las etiquetas de cada eje para evitar solapamientos
    fig.for_each_xaxis(lambda axis: axis.update(tickangle=0))
    fig.for_each_yaxis(lambda axis: axis.update(tickangle=90))
    
    fig.show()

def matriz_correlacion(df):
    """
    Función que recibe un DataFrame de pandas y genera una matriz de correlación
    visualizada con un heatmap de Plotly.
    """
    # Seleccionar solo columnas numéricas
    df_numerico = df.select_dtypes(include=['number'])

    if df_numerico.shape[1] < 2:
        print("El DataFrame debe tener al menos dos columnas numéricas para calcular la correlación.")
        return
    
    # Calcular la matriz de correlación
    corr_matrix = df_numerico.corr()

    # Crear el heatmap con Plotly
    fig = px.imshow(
        corr_matrix,
        text_auto=".2f",  # Muestra valores con 2 decimales
        labels=dict(color="Correlación"),
        title="Matriz de Correlación",
        color_continuous_scale="RdBu_r"  # Escala de colores para correlaciones positivas y negativas
    )

    # Ajustar la visualización
    fig.update_layout(width=700, height=600)
    fig.show()

    """
    Función que recibe un DataFrame de pandas y genera un análisis bivariante para
    las columnas categóricas, mostrando para cada par una tabla de contingencia
    en forma de heatmap agrupados de dos en dos en una misma figura.
    
    Parámetros:
      - df: DataFrame de pandas.
      - excluir: lista de columnas a excluir del análisis (opcional).
    """
    # Seleccionar columnas categóricas
    columnas_categoricas = df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Excluir las columnas indicadas (si las hay)
    if excluir is not None:
        columnas_categoricas = [col for col in columnas_categoricas if col not in excluir]
    
    if len(columnas_categoricas) < 2:
        print("El DataFrame debe tener al menos dos columnas categóricas (después de excluir) para realizar un análisis bivariante.")
        return
    
    # Lista para almacenar la información de cada par (columna1, columna2, tabla de contingencia)
    pares = []
    
    # Generar los pares de columnas y sus respectivas tablas de contingencia
    for i in range(len(columnas_categoricas)):
        for j in range(i+1, len(columnas_categoricas)):
            col1 = columnas_categoricas[i]
            col2 = columnas_categoricas[j]
            print(f"Analizando relación entre '{col1}' y '{col2}'")
            
            # Copiar y reemplazar los valores nulos por '?'
            df_temp = df[[col1, col2]].copy()
            df_temp[col1] = df_temp[col1].fillna("?")
            df_temp[col2] = df_temp[col2].fillna("?")
            
            # Generar la tabla de contingencia
            tabla_contingencia = pd.crosstab(df_temp[col1], df_temp[col2], dropna=False)
            
            if tabla_contingencia.empty:
                print(f"La tabla de contingencia para {col1} y {col2} está vacía.")
                continue
            
            pares.append((col1, col2, tabla_contingencia))
    
    # Agrupar los heatmaps de 2 en 2
    por_figura = 2
    for idx in range(0, len(pares), por_figura):
        grupo = pares[idx:idx+por_figura]
        ncols = len(grupo)
        
        # Crear una figura con subplots
        fig = make_subplots(rows=1, cols=ncols,
                            subplot_titles=[f"Tabla de contingencia: {p[0]} vs {p[1]}" for p in grupo])
        
        # Agregar cada heatmap al subplot correspondiente
        for k, (col1, col2, tabla_contingencia) in enumerate(grupo, start=1):
            # Extraer datos para el heatmap
            z = tabla_contingencia.values
            x = list(tabla_contingencia.columns)
            y = list(tabla_contingencia.index)
            
            heatmap = go.Heatmap(
                z=z,
                x=x,
                y=y,
                text=z,
                texttemplate="%{text}",
                colorscale='Viridis'
            )
            fig.add_trace(heatmap, row=1, col=k)
            # Actualizar etiquetas de cada subplot
            fig.update_xaxes(title_text=col2, row=1, col=k)
            fig.update_yaxes(title_text=col1, row=1, col=k)
        
        fig.update_layout(height=500, width=600*ncols, title_text="Heatmaps de Tablas de Contingencia (2 en 2)")
        fig.show()


    
    # Iterar sobre cada par de variables categóricas y graficar la tabla de contingencia
    for i in range(len(columnas_categoricas)):
        for j in range(i+1, len(columnas_categoricas)):
            col1 = columnas_categoricas[i]
            col2 = columnas_categoricas[j]

            print(f"Analizando relación entre '{col1}' y '{col2}'")
            
            # Copiar y reemplazar los valores nulos por un símbolo (por ejemplo, '?')
            df_temp = df[[col1, col2]].copy()
            df_temp[col1] = df_temp[col1].fillna("?")
            df_temp[col2] = df_temp[col2].fillna("?")
            
            # Generar la tabla de contingencia, sin eliminar categorías nulas
            tabla_contingencia = pd.crosstab(df_temp[col1], df_temp[col2], dropna=False)
            
            if tabla_contingencia.empty:
                print(f"La tabla de contingencia para {col1} y {col2} está vacía.")
                continue
            
            # Crear el heatmap utilizando plotly.express.imshow
            fig = px.imshow(
                tabla_contingencia,
                text_auto=True,
                labels=dict(x=col2, y=col1, color="Cuenta"),
                title=f"Tabla de contingencia: {col1} vs {col2}"
            )
            fig.update_layout(width=600, height=500)
            fig.show()

def analisis_categorico(df, excluir=None):
    """
    Función que recibe un DataFrame de pandas y genera un análisis bivariante para
    las columnas categóricas, mostrando para cada par una tabla de contingencia
    en forma de heatmap agrupados de dos en dos en una misma figura.
    
    Parámetros:
      - df: DataFrame de pandas.
      - excluir: lista de columnas a excluir del análisis (opcional).
    """
    # Seleccionar columnas categóricas
    columnas_categoricas = df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Excluir las columnas indicadas (si las hay)
    if excluir is not None:
        columnas_categoricas = [col for col in columnas_categoricas if col not in excluir]
    
    if len(columnas_categoricas) < 2:
        print("El DataFrame debe tener al menos dos columnas categóricas (después de excluir) para realizar un análisis bivariante.")
        return
    
    # Lista para almacenar la información de cada par (columna1, columna2, tabla de contingencia)
    pares = []
    
    # Generar los pares de columnas y sus respectivas tablas de contingencia
    for i in range(len(columnas_categoricas)):
        for j in range(i+1, len(columnas_categoricas)):
            col1 = columnas_categoricas[i]
            col2 = columnas_categoricas[j]
            
            # Copiar y reemplazar los valores nulos por '?'
            df_temp = df[[col1, col2]].copy()
            df_temp[col1] = df_temp[col1].fillna("?")
            df_temp[col2] = df_temp[col2].fillna("?")
            
            # Generar la tabla de contingencia
            tabla_contingencia = pd.crosstab(df_temp[col1], df_temp[col2], dropna=False)
            
            if tabla_contingencia.empty:
                print(f"La tabla de contingencia para {col1} y {col2} está vacía.")
                continue
            
            pares.append((col1, col2, tabla_contingencia))
    
    # Agrupar los heatmaps de 2 en 2
    por_figura = 2
    for idx in range(0, len(pares), por_figura):
        grupo = pares[idx:idx+por_figura]
        ncols = len(grupo)
        
        # Crear una figura con subplots
        fig = make_subplots(rows=1, cols=ncols,
                            subplot_titles=[f"Tabla de contingencia: {p[0]} vs {p[1]}" for p in grupo])
        
        # Agregar cada heatmap al subplot correspondiente
        for k, (col1, col2, tabla_contingencia) in enumerate(grupo, start=1):
            # Extraer datos para el heatmap
            z = tabla_contingencia.values
            x = list(tabla_contingencia.columns)
            y = list(tabla_contingencia.index)
            
            heatmap = go.Heatmap(
                z=z,
                x=x,
                y=y,
                text=z,
                texttemplate="%{text}",
                colorscale='Viridis'
            )
            fig.add_trace(heatmap, row=1, col=k)
            # Actualizar etiquetas de cada subplot
            fig.update_xaxes(title_text=col2, row=1, col=k)
            fig.update_yaxes(title_text=col1, row=1, col=k)
        
        fig.update_layout(height=500, width=600*ncols, title_text="Heatmaps de Tablas de Contingencia (2 en 2)")
        fig.show()



In [ ]:
analisis_numerico(df)

In [ ]:
matriz_correlacion(df)

In [ ]:
analisis_categorico(df, excluir=['encounter_id', 'patient_nbr'])

In [ ]:
import pandas as pd
import plotly.express as px

# Assuming df is your DataFrame
# df = pd.read_csv('your_dataset.csv')

# Get the value counts of patient_nbr
patient_counts = df['patient_nbr'].value_counts().reset_index()
patient_counts.columns = ['patient_nbr', 'counts']

patient_counts

In [ ]:
# df con solo pacientes que tienen pacient_nbr 23199021
filtered_df = df[df['patient_nbr'] == '23199021']

In [ ]:
keeping = ['encounter_id', 'patient_nbr', 'number_emergency', 'number_inpatient', 'number_outpatient', 'time_in_hospital', 'admission_type_id']

filtered_df[keeping]